# Arricchimento dati ANAC con IndicePA e OpenConsip

#### L'obiettivo di questa notebook e' arricchire i dati sui contratti pubblici raccolti da [anac legge 190](https://dati.anticorruzione.it/#/l190/2018) con i dati pubblicati sui dettagli delle PA da [indice pa](http://www.indicepa.gov.it/documentale/index.php).

## IndicePA 


Questa sezione arricchisce i dati delle stazioni appaltanti censite in Anac con informazioni presenti negli open data di IndicePA.

L'output di questa sezione sono i seguenti file:
- *indicePA.tsv*, contenente i dati estratti da IndicePA per le stazioni appaltanti in analisi;
- *otherPA.tsv*, contenente i codici fiscali delle stazioni appaltanti non censite in IndicePA;

[IndicePA](http://www.indicepa.gov.it/documentale/index.php) ragruppa le Pubbliche Amministrazioni (PA) in 3 categorie:

1. **AMMINISTRAZIONI**: informazioni di sintesi sull'Ente: indirizzo postale, codice fiscale, logo, responsabile e riferimenti telematici (sito web istituzionale, indirizzi di posta elettronica);

2. **Aree Organizzative Omogenee (AOO)**: ; informazioni sugli uffici di protocollo (Aree Organizzative Omogenee - AOO).

3. **Unità Organizzative (OU)**:informazioni sulla struttura organizzativa e gerarchica e sui singoli uffici (Unità Organizzative - UO), corredate con informazioni di dettaglio;

Per ogni categoria, IndicePA associa differenti informazioni. Per maggiori dettagli vedi [qui](http://www.indicepa.gov.it/public-services/docs-read-service.php?dstype=FS&filename=Metadati_Open_Data.pdf)

In particolare, il seguente notebook estrae dal file *lista_pa.json* tutti i codici fiscali (cf) delle PA censite da Anac. Ogni cf viene quindi ricercato nei file *data/indicePA/serv_fatt.txt* (PEC-CECPAC)  e *data/indicePA/amministrazioni.txt* (opendata contenente informazioni su tutte le PA di tipo amministrazioni) dal quale vengono estratti i seguenti attributi:

* Codice Fiscale
* Regione
* Provincia
* Comune 
* Indirizzo
* Tipologia Istat
* tipologia_amm

Il file *lista_pa.json* è generato dalla seguente chiamata rest:
> curl -H "Content-Type: application/json" -X POST -d  '{"anno":"2018","codiceFiscaleAmministrazione":"","denominazioneAmministrazione":"","identificativoComunicazione":""}' https://dati.anticorruzione.it/rest/legge190/ricerca\?max\=200000\&start\=0 -o lista_pa.json

 
L'integrazione del dataset ANAC con il dataset salvato in *indicePA.tsv* è realizzata nel notebook  **2_Prima Analisi esplorativa ANAC** alla sezione *Integrazione dati ANAC con IndicePA*.

## Note

1. Ci sono 1118 Codici Fiscali censiti da Anac che non son presenti in IndicePA. Nella maggior parte dei casi si tratta di SPA (e.g. 10200211000, 00222610289, 95107460107, etc.).

2. Ci sono 4 PA i cui Codici Fiscali non corripondono con nessun *codice amministrazione* (i.e. 00491510822, 00251300067, 01607380670, 00431990068). 



In [1]:
import pandas as pd
import numpy
import csv

In [2]:
#file scaricabili dal seguente link http://www.indicepa.gov.it/documentale/n-opendata.php
file_amministrazioni = "data/indicePA/amministrazioni.txt"
file_aoo = "data/indicePA/aoo.txt"
file_ou = "data/indicePA/ou.txt"
file_PEC_CECPAC = "data/indicePA/pec.txt"
file_serv_fatt = "data/indicePA/serv_fatt.txt"
file_lista_anac_pa = "data/anac/lista_pa.json"

In [3]:
#df_amm = pd.read_csv(file_amministrazioni, sep="\t", index_col="cod_amm", dtype=object)
df_amm = pd.read_csv(file_amministrazioni, sep="\t", dtype=object)
df_amm = df_amm[[ 'cod_amm','Cf','Comune', 'Provincia', 'Regione', 'Indirizzo', 'tipologia_istat', 'tipologia_amm', 'acronimo']]
df_amm.head()


,cod_amm,Cf,Comune,Provincia,Regione,Indirizzo,tipologia_istat,tipologia_amm,acronimo
0,054,03301860544,Perugia,PG,Umbria,"Via Guerriero Guerra, 21",Aziende Sanitarie Locali,Pubbliche Amministrazioni,
1,055,01499590550,Terni,TR,Umbria,"Viale Donato Bramante, 37",Aziende Sanitarie Locali,Pubbliche Amministrazioni,ASLUMBRIA2
2,056,01455570562,Viterbo,VT,Lazio,"Via Enrico Fermi, 15",Aziende Sanitarie Locali,Pubbliche Amministrazioni,NaN
3,058,04733471009,Tivoli,RM,Lazio,Via Acquaregna 1/15,Aziende Sanitarie Locali,Pubbliche Amministrazioni,NaN
4,080,02638720801,Reggio Calabria,RC,Calabria,"Via Sant'Anna II Tronco, 18/p",Aziende Sanitarie Locali,Pubbliche Amministrazioni,ASPRC


In [4]:
grouped_amm = df_amm[["tipologia_istat", "tipologia_amm", "Cf"]].groupby(["tipologia_istat", "tipologia_amm"]).agg(["count"])
grouped_amm.head()

,,Cf
,,count
tipologia_istat,tipologia_amm,
Agenzie Fiscali,Pubbliche Amministrazioni,4
Agenzie Regionali Sanitarie,Pubbliche Amministrazioni,12
Agenzie Regionali e Provinciale per la Rappresentanza Negoziale,Pubbliche Amministrazioni,1
Agenzie Regionali per le Erogazioni in Agricoltura,Pubbliche Amministrazioni,9
Agenzie ed Enti Regionali del Lavoro,Pubbliche Amministrazioni,9


In [5]:
df_aoo = pd.read_csv(file_aoo, sep="\t")
print(df_aoo.columns)
df_aoo = df_aoo[['cod_amm', 'cod_aoo', 'Comune', 'Provincia', 'Regione', 'Indirizzo']]
df_aoo.head()

Index(['cod_amm', 'cod_aoo', 'des_aoo', 'dat_istituz', 'Comune', 'Cap',
       'Provincia', 'Regione', 'Indirizzo', 'Tel', 'nome_resp', 'cogn_resp',
       'mail_resp', 'tel_resp', 'Fax', 'mail1', 'tipo_mail1', 'mail2',
       'tipo_mail2', 'mail3', 'tipo_mail3'],
      dtype='object')


,cod_amm,cod_aoo,Comune,Provincia,Regione,Indirizzo
0,055,ASLUMBRIA2,Terni,TR,Umbria,V.le D. Bramante n. 37
1,058,ABS,Tivoli,RM,Lazio,Via Acquaregna 1/15
2,058,DIP-Prevenzione,Tivoli,RM,Lazio,Via Acquaregna 1/15
3,092,AOO-AOB,Cagliari,CA,Sardegna,"Piazzale A. Ricchi, 1"
4,093,protgen,Pordenone,PN,Friuli Venezia Giulia,"Via Vecchia Ceramica, 1"


In [6]:
df_ou = pd.read_csv(file_ou, sep="\t")
print(df_ou.columns)
df_ou = df_ou[['cod_amm', 'cod_aoo', 'cod_ou', 'comune', 'provincia', 'Regione', 'Indirizzo']]

df_ou = df_ou.rename(columns={'comune': 'Comune', 'provincia': 'Provincia'})
df_ou.head()

Index(['cod_ou', 'cod_aoo', 'des_ou', 'comune', 'Cap', 'provincia', 'Regione',
       'Indirizzo', 'Tel', 'nome_resp', 'cogn_resp', 'mail_resp', 'tel_resp',
       'cod_amm', 'cod_ou_padre', 'Fax', 'cod_uni_ou', 'mail1', 'tipo_mail1',
       'mail2', 'tipo_mail2', 'mail3', 'tipo_mail3'],
      dtype='object')


/home/fabiana/miniconda3/envs/anac/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cod_amm,cod_aoo,cod_ou,Comune,Provincia,Regione,Indirizzo
0,c_f335,aooac,serfin,Moncalieri,TO,Piemonte,Piazza Vittorio Emanuele II snc
1,c_f335,aooac,sertec,Moncalieri,TO,Piemonte,Piazza Vittorio Emanuele II snc
2,c_f335,aooac,comlav,Moncalieri,TO,Piemonte,Piazza Vittorio Emanuele II snc
3,avvst,adsrm,adsrm01,Roma,RM,Lazio,"Via dei Portoghesi, 12"
4,avvst,adsrm,adsrm03,Roma,RM,Lazio,"Via del Clementino, 91/A Roma"


In [7]:
#df_pec = pd.read_csv(file_PEC_CECPAC, sep="\t", index_col="cod_amm")
df_pec = pd.read_csv(file_PEC_CECPAC, sep="\t")
print(df_pec.columns)
df_pec = df_pec[['cod_amm','Tipo', 'Tipologia Istat']]
df_pec.head()

Index(['cod_amm', 'Descrizione', 'Tipo', 'Tipologia Istat', 'Regione',
       'Provincia', 'Comune', 'Mail', 'Tipo Mail'],
      dtype='object')


,cod_amm,Tipo,Tipologia Istat
0,abam_043,SERVAMM,"Istituzioni per l'Alta Formazione Artistica, M..."
1,abfitlpb,SERVAMM,Autorita' di Bacino
2,abspa,SERVAMM,Gestori di Pubblici Servizi
3,ac_va,SERVAMM,Automobile Club Federati ACI
4,aimcr,SERVAMM,Aziende Pubbliche di Servizi alla Persona


In [8]:
df_serv_fatt = pd.read_csv(file_serv_fatt, sep="\t", dtype=object)
print(df_serv_fatt.columns)
#df_serv_fatt = df_serv_fatt[[ 'cod_amm','Cf','Comune', 'Provincia', 'Regione', 'Indirizzo']]
df_serv_fatt = df_serv_fatt.rename(columns={'cf': 'Cf'})
df_serv_fatt.head()

Index(['cod_amm', 'cod_ou', 'cod_uni_ou', 'des_ou', 'regione', 'provincia',
       'comune', 'indirizzo', 'cap', 'cf', 'dt_verifica_cf', 'data_avvio_sfe'],
      dtype='object')


,cod_amm,cod_ou,cod_uni_ou,des_ou,regione,provincia,comune,indirizzo,cap,Cf,dt_verifica_cf,data_avvio_sfe
0,054,Uff_eFatturaPA,UF9FAJ,Uff_eFatturaPA,Umbria,PG,Perugia,"Via Guerriero Guerra, 21",06127,03301860544,2015-04-01,2015-03-31
1,055,SS01,Y0BN4H,Servizio economico finanziario Sanitario,Umbria,TR,Terni,Via Bramante 37,05100,01499590550,2015-04-02,2015-03-31
2,055,SS02,H2J28R,Servizio economico finanziario Servizi Sociali,Umbria,TR,Terni,Via Bramante 37,05100,01499590550,2015-03-31,2015-03-31
3,055,Uff_eFatturaPA,UF7K0M,Uff_eFatturaPA,Umbria,TR,Terni,"Viale Donato Bramante, 37",05100,01499590550,2015-03-31,2015-03-31
4,056,Uff_eFatturaPA,UF9IAK,Uff_eFatturaPA,Lazio,VT,Viterbo,"Via Enrico Fermi, 15",01100,01455570562,2015-04-08,2015-03-31


In [9]:
print("Dimensione file amministrazioni.txt", df_amm.shape)
print("Dimensione file aoo.txt", df_aoo.shape)
print("Dimensione file ou.txt", df_ou.shape)
print("Dimensione file pec.txt", df_pec.shape)

Dimensione file amministrazioni.txt (22648, 9)
Dimensione file aoo.txt (26888, 6)
Dimensione file ou.txt (92573, 7)
Dimensione file pec.txt (112684, 3)


### Arricchimento dati da *lista_pa.json* 

In [10]:
import json
from pprint import pprint
cf_set_serv_fatt = set(df_serv_fatt.Cf.values)
cf_set_amm = set(df_amm.Cf.values)
data = json.load(open(file_lista_anac_pa))

In [11]:
with open("data/indicePA/indicePA.tsv", 'w') as f_indice_pa , open("data/indicePA/otherPA.tsv", 'w') as f_other:
    writer_indice_pa = csv.writer(f_indice_pa, delimiter ='\t')
    writer_other_pa = csv.writer(f_other, delimiter ='\t')

    count = 0
    res = ["cf", "cod_amm", "regione", "provincia", "comune", "indirizzo", "tipologia_istat", "tipologia_amm"]
    writer_indice_pa.writerow(res)
    writer_other_pa.writerow(["cf"])

    for pa in data["result"]:
        esito = pa["esitoUltimoTentativoAccessoUrl"]
        if esito == "successo":
            cf = pa["codiceFiscale"]

            if cf in cf_set_amm:
                try:
                    cod_amm = df_amm.loc[df_amm['Cf'] == cf].iloc[0]['cod_amm']
                    take0 = df_amm.loc[df_amm['cod_amm'] == cod_amm].iloc[0]
                    regione = take0['Regione'].replace("\t", "")
                    provincia = str(take0['Provincia']).replace("\t", "")
                    comune = take0['Comune'].replace("\t", "")
                    indirizzo = take0['Indirizzo'].replace("\t", "")
                    tipologia_istat = take0['tipologia_istat'].replace("\t", "")
                    tipologia_amm = take0['tipologia_amm'].replace("\t", "")

                    res = [cf, cod_amm, regione, provincia, comune, indirizzo, tipologia_istat, tipologia_amm]
                    writer_indice_pa.writerow(res)
                except: # catch *all* exceptions
                    print("CF in df_amm",cf)
            elif cf in cf_set_serv_fatt:
                try:
                    cod_amm = df_serv_fatt.loc[df_serv_fatt['Cf'] == cf].iloc[0]['cod_amm']
                    take0 = df_amm.loc[df_amm['cod_amm'] == cod_amm].iloc[0]
                    regione = take0['Regione'].replace("\t", "")
                    provincia = str(take0['Provincia']).replace("\t", "")
                    comune = take0['Comune'].replace("\t", "")
                    indirizzo = take0['Indirizzo'].replace("\t", "")
                    tipologia_istat = take0['tipologia_istat'].replace("\t", "")
                    tipologia_amm = take0["tipologia_amm"].replace("\t", "")
                    res = [cf, cod_amm, regione, provincia, comune, indirizzo, tipologia_istat, tipologia_amm]
                    writer_indice_pa.writerow(res)
                except: # catch *all* exceptions
                    #e = sys.exc_info()[0]
                    print("CF in df_serv_fatt",cf)
            else:
                #print(cf, " is not present")
                count = count + 1
                writer_other_pa.writerow([cf])
                #if(count % 100 == 0):
                    #print(cf)

    print("Totale cf non presenti in IndicePA: ", count)
    f_indice_pa.flush()   
    f_other.flush()

Totale cf non presenti in IndicePA:  1148


### Link Utili

[1] [Specifiche ANAC](https://www.anticorruzione.it/portal/rest/jcr/repository/collaboration/Digital%20Assets/anacdocs/Servizi/ServiziOnline/AdempimentoLegge190/Specifiche%20Tecniche%20Legge%20190%20v1.2_finale.pdf)

[2] [](http://portaletrasparenza.anticorruzione.it/Microstrategy/asp/Main.aspx)

[3] [Open data IndicePA](http://www.indicepa.gov.it/documentale/n-opendata.php)

## OpenConsip

Questa sezione aggrega tutti i fornitori forniti da [OpenConsip](http://dati.consip.it/dataset/dataset-fornitori) per gli anni 2016, 2017, 2018 (unici anni ad oggi disponibili). 

Il dataset è caratterizzato da 110.251 fornitori distinti (per partita iva).

L'integrazione del dataset ANAC con il dataset dei fornitori in OpenConsip è realizzata nel notebook  **2_Prima Analisi esplorativa ANAC** alla sezione *Integrazione dati ANAC con OpenConsip*.



In [12]:
import urllib.request

In [13]:
#url fornitori
url_fornitori = ["http://dati.consip.it/download/9a32cc52-05c9-4a72-95e5-a79709f33cac/fornitori2018.xlsx",
                "http://dati.consip.it/download/79724110-d914-4f0a-b011-0b6443028657/fornitori2017.xlsx",
                "http://dati.consip.it/download/f476dccf-d60a-4301-b757-829b3e030ac6/fornitori2016.xlsx"
                ]
data = []
for u in url_fornitori:
    data.append(pd.read_excel(u))

In [14]:
df_fornitori = pd.concat(data)
df_fornitori.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252843 entries, 0 to 72997
Data columns (total 12 columns):
#Partita_Iva               252843 non-null object
Ragione_Sociale            252843 non-null object
Forma_Societaria           252732 non-null object
Nazione_Sede_legale        252833 non-null object
Regione_Sede_legale        252807 non-null object
Provincia_Sede_legale      252807 non-null object
Comune_Sede_legale         252807 non-null object
Indirizzo_Sede_legale      252823 non-null object
Numero_Aggiudicazioni      252843 non-null int64
Numero_Abilitazioni        252843 non-null int64
Numero_Transazioni         252843 non-null int64
Numero_Contratti_attivi    252843 non-null int64
dtypes: int64(4), object(8)
memory usage: 25.1+ MB


In [15]:
df_fornitori.drop_duplicates(["#Partita_Iva"], inplace=True)
print('Dimensione Dataset senza duplicati: ',df_fornitori.shape)
df_fornitori.to_pickle("data/consip/fornitori.pickle")

Dimensione Dataset senza duplicati:  (110251, 12)
